In [8]:
import json
import os
import pandas as pd
import numpy as np

labels = ["fs", "fs-em", "zf", "zf-em", "x"]
label_indices = {
    "fs": 0,
    "fs-em": 1,
    "zf": 2,
    "zf-em": 3,
    "x": 4,
}

all_paths = []
for path in os.listdir("./data"):
    if not os.path.isdir(path):
        all_paths.append("./data/" + path)
        
all_data = []
for path in all_paths:
    print(path)
    with open(path, "r") as f:
        data = json.load(f)
        all_data.append(data)

chosen_matrix = np.zeros((len(labels), len(labels)))
count_matrix = np.zeros((len(labels), len(labels)))
for data in all_data:
    for task in data:
        left_id = label_indices[task["left"]]
        right_id = label_indices[task["right"]]
        chosen_matrix[left_id, right_id] += 1 if task["choice"] == 0 else 0
        count_matrix[left_id, right_id] += 1
        
prob_matrix = chosen_matrix / count_matrix
print(chosen_matrix)
print(count_matrix)
print(prob_matrix)

./data/002.json
./data/003.json
./data/004.json
./data/005.json
[[ 2.  7. 10.  3.  7.]
 [ 4.  2. 13.  7.  6.]
 [ 1.  0.  2.  4.  0.]
 [ 1.  2.  4.  1.  1.]
 [ 4.  3.  9. 14.  1.]]
[[ 5. 10. 10.  5. 12.]
 [ 8.  5. 14.  9.  9.]
 [15.  7.  5. 12. 10.]
 [ 9. 10.  6.  5.  5.]
 [10.  9. 10. 15.  5.]]
[[0.4        0.7        1.         0.6        0.58333333]
 [0.5        0.4        0.92857143 0.77777778 0.66666667]
 [0.06666667 0.         0.4        0.33333333 0.        ]
 [0.11111111 0.2        0.66666667 0.2        0.2       ]
 [0.4        0.33333333 0.9        0.93333333 0.2       ]]
